In [1]:
import json
import pickle
import csv

import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
import tensorflow as tf
from sklearn.model_selection import train_test_split,KFold, GroupKFold,StratifiedKFold

import tensorflow.keras as keras

import gc
import matplotlib.pyplot as plt
import os
import sys

from tqdm import tqdm

import numpy as np
import pandas as pd
import gzip
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
tf.__version__

'2.4.1'

In [3]:
LOSS_WGTS = [0.3, 0.3, 0.3, 0.05, 0.05] #column weights, need to sum up to 1


DIST_NEW = True
DIST_NEW2 = True

BBP = True
BBP1 = True
BBP2 = True
BBP3 = True
BBP4 = True

BBP_TOTAL = BBP+BBP1+BBP2+BBP3+BBP4*4

# This will tell us the columns we are predicting
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']

In [4]:
df = pd.read_csv('../../../DeepDeg3/data/Round6_3/nov16_last57_CDS.csv')
df.head()

,id,title,name,sequence,CAI,GC content,SUP vienna,SUP first 14 vienna,SUP first 30 vienna,punp_vec vienna,...,MFE Struct EternaFold,bprna_string,degscore_vec,degscore,MLD,n_hairpins,n_3WJs,n_4WJs,n_5WJs_up,hp_3WJ_ratio
0,10421216,Mod of wateronthemoon's OV:nanofl 6,Merida,AUGGCCGUCUACCCCUACGACGUGCCCGACUACGCCGGCUACCCCU...,0.890295,59.742351,195.361171,5.711359,16.950537,[9.98375821e-01 6.18258391e-01 2.89686821e-03 ...,...,..(((((((.........)))).))).......(((((((.(((((...,EESSSSSSSHHHHHHHHHSSSSBSSSEEEEEEESSSSSSSMSSSSS...,[ 0.747 0.393 0.36 0.242 -0.1 0.11 0....,177.064,78,20,5,1,3,3.999999
1,10421202,phoning it in 5,cynwulf28,AUGGCUGUAUAUCCGUAUGAUGUGCCCGAUUAUGCUGGGUACCCAU...,0.754168,51.529791,191.430001,3.476410,10.565406,[9.87866639e-01 9.84536094e-01 3.36349718e-03 ...,...,..(((((.((((((((.....(((((((.......))))))).(((...,EESSSSSBSSSSSSSSMMMMMSSSSSSSHHHHHHHSSSSSSSMSSS...,[ 5.96e-01 3.17e-01 3.28e-01 1.21e-01 1.50...,188.019,118,14,8,2,0,1.750000
2,10421119,Mod of DigitalEmbrace's Curevac 4 Mod 2-Kaggl...,Merida,AUGGCUGUUUACCCUUACGAUGUUCCCGACUACGCCGGGUACCCGU...,0.786049,59.742351,179.628614,7.173642,15.450940,[9.74989208e-01 9.99191615e-01 5.90287475e-03 ...,...,..(((((((............(((((((.(...(.(((((((.(((...,EESSSSSSSMMMMMMMMMMMMSSSSSSSISIIISISSSSSSSISSS...,[ 0.6 0.324 0.326 0.066 0.106 0.075 0....,170.490,96,17,6,1,2,2.833333
3,10421010,phoning it in 4,cynwulf28,AUGGCCGUGUAUCCUUAUGAUGUUCCAGAUUAUGCCGGCUACCCCU...,0.778542,49.758454,218.053576,5.990581,8.633370,[9.73084676e-01 9.34549226e-01 9.55890625e-01 ...,...,...((((((((.(((..((((((((((((((((.(((((..........,EEESSSSSSSSBSSSIISSSSSSSSSSSSSSSSISSSSSIIIIIII...,[ 7.24000000e-01 3.49000000e-01 4.86000000e-...,206.416,61,16,4,1,2,3.999999
4,10420989,phoning it in 3,cynwulf28,AUGGCCGUUUACCCGUACGAUGUGCCUGACUACGCGGGCUAUCCGU...,0.797147,55.877617,213.959940,3.269031,6.952990,[7.92954489e-01 6.56017560e-01 1.23806781e-01 ...,...,(((((((((..((((((((....(((((......)))))....)))...,SSSSSSSSSIISSSSSSSSIIIISSSSSHHHHHHSSSSSIIIISSS...,[ 4.88000000e-01 1.36000000e-01 1.69000000e-...,191.271,90,18,6,1,2,3.000000


In [5]:
df.shape

(57, 26)

In [6]:
df.columns

Index(['id', 'title', 'name', 'sequence', 'CAI', 'GC content', 'SUP vienna',
       'SUP first 14 vienna', 'SUP first 30 vienna', 'punp_vec vienna',
       'mean bp prox vienna', 'SUP eternafold', 'SUP first 14 eternafold',
       'SUP first 30 eternafold', 'punp_vec eternafold',
       'mean bp prox eternafold', 'MFE Struct EternaFold', 'bprna_string',
       'degscore_vec', 'degscore', 'MLD', 'n_hairpins', 'n_3WJs', 'n_4WJs',
       'n_5WJs_up', 'hp_3WJ_ratio'],
      dtype='object')

In [7]:
#df['sequence'] = df['RNA']
df['structure'] = df['MFE Struct EternaFold']
df['bpRNA_string'] = df['bprna_string']
df['seq_length'] = df['sequence'].apply(lambda x: len(x))
original_ids = df['id'].values
#df['id'] = list(range(476))
df.sort_values(by='seq_length')

,id,title,name,sequence,CAI,GC content,SUP vienna,SUP first 14 vienna,SUP first 30 vienna,punp_vec vienna,...,degscore,MLD,n_hairpins,n_3WJs,n_4WJs,n_5WJs_up,hp_3WJ_ratio,structure,bpRNA_string,seq_length
0,10421216,Mod of wateronthemoon's OV:nanofl 6,Merida,AUGGCCGUCUACCCCUACGACGUGCCCGACUACGCCGGCUACCCCU...,0.890295,59.742351,195.361171,5.711359,16.950537,[9.98375821e-01 6.18258391e-01 2.89686821e-03 ...,...,177.064,78,20,5,1,3,3.999999e+00,..(((((((.........)))).))).......(((((((.(((((...,EESSSSSSSHHHHHHHHHSSSSBSSSEEEEEEESSSSSSSMSSSSS...,621
30,10418991,OV:nanofl 17,wateronthemoon,AUGGCCGUUUACCCAUACGAUGUUCCUGACUAUGCGGGCUAUCCCU...,0.746635,50.402576,256.460310,3.684035,10.040151,[2.79026532e-01 2.04186457e-01 5.50948566e-02 ...,...,221.975,109,12,4,3,0,2.999999e+00,((((((((...((((.((((....((((......((((...((......,SSSSSSSSMMMSSSSBSSSSMMMMSSSSIIIIIISSSSIIISSHHH...,621
31,10418986,OV:nanofl 16,wateronthemoon,AUGGCCGUGUAUCCAUACGACGUCCCGGAUUAUGCGGGGUACCCCU...,0.768023,58.454106,165.480017,3.106928,10.134007,[9.87866327e-01 9.84566923e-01 3.35897264e-03 ...,...,165.946,93,13,6,1,1,2.166666e+00,..(((((.((((((......(((((.((((((.(((((((((((((...,EESSSSSBSSSSSSMMMMMMSSSSSBSSSSSSISSSSSSSSSSSSS...,621
32,10418947,AB_Cynwulf_Mod_1,amybarish,AUGGCCGUGUACCCCUACGAUGUCCCGGACUACGCGGGGUAUCCGU...,0.765765,60.547504,161.584316,4.171178,8.989436,[9.99556577e-01 6.69514409e-01 7.45388498e-02 ...,...,162.560,93,18,5,2,2,3.599999e+00,..(.(((.((((...((((((((((((.......))))))))).))...,EESISSSBSSSSIIISSSSSSSSSSSSHHHHHHHSSSSSSSSSBSS...,621
33,10418790,161.1: mod of rna-key mod of linearfold beam 50,Omei,AUGGCCGUGUACCCCUACGAUGUCCCCGACUACGCCGGAUACCCGU...,0.775030,54.106280,123.898994,3.337207,4.406407,[9.87866160e-01 9.84567669e-01 3.36397490e-03 ...,...,160.273,125,14,6,1,1,2.333333e+00,..(((((.((((((((((((.(((((((((.(((((((((.(((((...,EESSSSSBSSSSSSSSSSSSISSSSSSSSSISSSSSSSSSBSSSSS...,621
34,10418785,#MLD - 6 (skyblue),Eli Fisker,AUGGCGGUGUACCCCUACGACGUCCCGGACUACGCCGGCUACCCGU...,0.798828,61.352657,171.862960,4.091485,6.079088,[9.96126157e-01 9.94950918e-01 9.80200295e-01 ...,...,157.149,86,17,3,1,3,5.666665e+00,...(((((((.(((((((((.((((.((((.((((((.((((((((...,EEESSSSSSSMSSSSSSSSSISSSSISSSSBSSSSSSBSSSSSSSS...,621
35,10418756,Kaggle - Omei 1 DG2:158.7 5'UTR 20 unpaired,Eli Fisker,AUGGCCGUGUACCCGUACGAUGUGCCCGACUACGCCGGGUAUCCGU...,0.765034,56.521739,123.358715,9.531709,15.169521,[7.23024746e-01 6.82627299e-01 6.26910812e-01 ...,...,155.470,228,18,3,3,2,5.999998e+00,....................((.(((((....((((((((....((...,EEEEEEEEEEEEEEEEEEEESSISSSSSIIIISSSSSSSSIIIISS...,621
36,10418713,Omei 1 DG2:158.7 5'UTR 20 unpaired (Omei),Eli Fisker,AUGGCCGUGUACCCGUACGAUGUGCCCGACUACGCCGGGUACCCGU...,0.775420,57.809984,123.166198,9.528626,15.165877,[7.23273160e-01 6.82777538e-01 6.26405180e-01 ...,...,153.749,226,17,4,2,2,4.249999e+00,....................((.(((((....((((((((....((...,EEEEEEEEEEEEEEEEEEEESSISSSSSIIIISSSSSSSSIIIISS...,621
37,10418630,Kaggle - Mod of Eli's BugacMan's Worm refined...,Eli Fisker,AUGGCUGUCUAUCCUUAUGACGUUCCGGACUACGCCGGAUAUCCGU...,0.739918,56.682770,122.451023,3.815657,10.937418,[9.99987573e-01 7.70423417e-01 6.97321314e-03 ...,...,151.659,251,11,7,1,0,1.571428e+00,.(((.((((((((.(((.(((((((.......((((((((....((...,ESSSISSSSSSSSBSSSISSSSSSSIIIIIIISSSSSSSSIIIISS...,621
38,10418559,Phone Submission 1,cynwulf28,AUGGCCGUUUACCCAUACGAUGUUCCUGACUAUGCGGGCUAUCCCU...,0.774912,52.818035,243.743997,3.674213,10.043126,[2.87377655e-01 2.11512151e-01 5.62282341e-02 ...,...,220.690,108,13,5,3,0,2.599999e+00,((((((((...((((.((((....((((......((((...((......,SSSSSSSSMMMSSSSBSSSSMMMMSSSSIIIIIISSSSIIISSHHH...,621


In [8]:
def pandas_list_to_array(df):
    """
    Input: dataframe of shape (x, y), containing list of length l
    Return: np.array of shape (x, l, y)
    """
    
    return np.transpose(
        np.array(df.values.tolist()),
        (0, 2, 1)
    )


def calc_neighbor(d, dim, n):
    lst_x,lst_y = np.where(d==n)
    for c, x in enumerate(lst_x):
        y = lst_y[c]    
        if x+1<dim:
            d[x+1,y] = min(d[x+1,y], n+1)
        if y+1<dim:
            d[x,y+1] = min(d[x,y+1], n+1)
        if x-1>=0:
            d[x-1,y] = min(d[x-1,y], n+1)
        if y-1>=0:
            d[x,y-1] = min(d[x,y-1], n+1)
    return d
            

def get_distance_matrix_2d(Ss):
    Ds = []
    n = Ss.shape[0]
    dim = Ss.shape[1]
    for i in range(n):
        s = Ss[i,:,:,0]
        d = 10+np.zeros_like(s)
        d[s==1] = 1
        for i in range(dim):
            d[i,i] = 0
        for x in range(0, 9):
            d = calc_neighbor(d, dim, x)
        Ds.append(d)
    Ds =  np.array(Ds) + 1
    Ds = 1/Ds
    Ds = Ds[:, :,:, None]
    
    Dss = []
    for i in [1, 2, 4]:
        Dss.append(Ds ** i)
    Ds = np.stack(Dss, axis = 3)
    return Ds[:,:,:,:,0]

In [9]:
# loss functions
def MCRMSE(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=(1))
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)


from tensorflow.keras import losses

class MSE(losses.MeanSquaredError):
    def __init__(self, *args, **kwargs):
        losses.MeanSquaredError.__init__(self, *args, **kwargs)

    def __call__(self, y_true, y_pred, sample_weight=None):
        y_true = tf.where(tf.math.is_nan(y_true), y_pred, y_true)
        
        temp = losses.MeanSquaredError.__call__(self, y_true[:, :, 0], y_pred[:, :, 0], sample_weight=None)
        temp = tf.sqrt(temp+1e-12)
        temp = tf.tensordot(temp,sample_weight,1)/tf.reduce_sum(sample_weight)
        s = temp*LOSS_WGTS[0]
#         s = tf.sqrt(temp)*LOSS_WGTS[0]
        for i in range(1,5):
            temp = losses.MeanSquaredError.__call__(self, y_true[:, :, i], y_pred[:, :, i], sample_weight=None)
            temp = tf.sqrt(temp+1e-12)
            temp = tf.tensordot(temp,sample_weight,1)/tf.reduce_sum(sample_weight)
#             s += tf.sqrt(temp)*LOSS_WGTS[i]
            s += (temp)*LOSS_WGTS[i]
            
        return s
    
    
from sklearn.metrics import mean_squared_error

def mean_squared_error1(y_true, y_pred, sample_weight):
    return np.sum((np.sqrt(np.mean((y_true-y_pred)**2, axis=1)))*sample_weight)/np.sum(sample_weight)

def MCRMSE_NAN_sample_wgt(y_true, y_pred, sample_weight=None, loss_cap=None):
    if loss_cap is not None:
        y_true_adj = np.minimum(np.maximum(y_true, y_pred-loss_cap), y_pred+loss_cap)
        return MCRMSE_NAN_sample_wgt(y_true_adj, y_pred, sample_weight=sample_weight, loss_cap=None)
    
    y_wgt = tf.ones_like(y_true)
    y_true = tf.where(tf.math.is_nan(y_true), y_pred, y_true)

    s = (mean_squared_error1(y_true[:, :, 0], y_pred[:, :, 0], sample_weight=sample_weight)/(tf.reduce_mean(y_wgt[:,:, 0])))*LOSS_WGTS[0]
    for i in range(1,5):
        s += (mean_squared_error1(y_true[:, :, i], y_pred[:, :, i], sample_weight=sample_weight)/(tf.reduce_mean(y_wgt[:,:, i])))*LOSS_WGTS[i]
    return s

def MCRMSE_NAN_sample_wgt_single(y_true, y_pred, sample_weight=None, loss_cap=None):
    if loss_cap is not None:
        y_true_adj = np.minimum(np.maximum(y_true, y_pred-loss_cap), y_pred+loss_cap)
        return MCRMSE_NAN_sample_wgt_single(y_true_adj, y_pred, sample_weight=sample_weight, loss_cap=None)
        
    y_wgt = tf.ones_like(y_true)
    y_true = tf.where(tf.math.is_nan(y_true), y_pred, y_true)

    s = (mean_squared_error1(y_true[:, :], y_pred[:, :], sample_weight=sample_weight)/(tf.reduce_mean(y_wgt[:,:])))
    return s


def MCRMSE_NAN(y_true, y_pred, wgt=LOSS_WGTS, loss_cap=None):
    return MCRMSE_NAN_sample_wgt(y_true, y_pred, sample_weight=tf.ones_like(y_true[:,0,0]), loss_cap=loss_cap)

In [10]:
# reverse inputs
def reverse_input(train_input):
    reverse = train_input[:, ::-1, :]
    return reverse

def reverse_BBP_3D(mat):
    return mat[:, ::-1, ::-1,:]

In [11]:
# from https://www.kaggle.com/xhlulu/openvaccine-simple-gru-model

def gru_layer(hidden_dim, dropout):
    return L.Bidirectional(
        L.GRU(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer='orthogonal')
    )


def lstm_layer(hidden_dim, dropout):
    return L.Bidirectional(
              L.LSTM(hidden_dim,dropout=dropout, return_sequences=True,kernel_initializer = 'orthogonal'))

In [12]:
# from https://www.kaggle.com/ragnar123/wavenet-gru-baseline

def wave_block(x, filters, kernel_size, n):
    dilation_rates = [2 ** i for i in range(n)]
    x = tf.keras.layers.Conv1D(filters = filters, 
                               kernel_size = 1,
                               padding = 'same')(x)
    res_x = x
    for dilation_rate in dilation_rates:
        tanh_out = tf.keras.layers.Conv1D(filters = filters,
                          kernel_size = kernel_size,
                          padding = 'same', 
                          activation = 'tanh', 
                          dilation_rate = dilation_rate)(x)
        sigm_out = tf.keras.layers.Conv1D(filters = filters,
                          kernel_size = kernel_size,
                          padding = 'same',
                          activation = 'sigmoid', 
                          dilation_rate = dilation_rate)(x)
        x = tf.keras.layers.Multiply()([tanh_out, sigm_out])
        x = tf.keras.layers.Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = tf.keras.layers.Add()([res_x, x])
    return res_x

In [13]:
# main model edited from https://www.kaggle.com/mrkmakr/covid-ae-pretrain-gnn-attn-cnn

def attention(x_inner, x_outer, n_factor, dropout):
    x_Q =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_inner)
    x_K =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_outer)
    x_V =  L.Conv1D(n_factor, 1, activation='linear', 
                  kernel_initializer='glorot_uniform',
                  bias_initializer='glorot_uniform',
                 )(x_outer)
    x_KT = L.Permute((2, 1))(x_K)
    res = L.Lambda(lambda c: K.batch_dot(c[0], c[1]) / np.sqrt(n_factor))([x_Q, x_KT])
    att = L.Lambda(lambda c: K.softmax(c, axis=-1))(res)
    att = L.Lambda(lambda c: K.batch_dot(c[0], c[1]))([att, x_V])
    return att

def multi_head_attention(x, y, n_factor, n_head, dropout):
    if n_head == 1:
        att = attention(x, y, n_factor, dropout)
    else:
        n_factor_head = n_factor // n_head
        heads = [attention(x, y, n_factor_head, dropout) for i in range(n_head)]
        att = L.Concatenate()(heads)
        att = L.Dense(n_factor, 
                      kernel_initializer='glorot_uniform',
                      bias_initializer='glorot_uniform',
                     )(att)
    x = L.Add()([x, att])
    x = L.LayerNormalization()(x)
    if dropout > 0:
        x = L.Dropout(dropout)(x)
    return x

def res(x, unit, kernel = 3, rate = 0.1):
    h = L.Conv1D(unit, kernel, 1, padding = "same", activation = None)(x)
    h = L.LayerNormalization()(h)
    h = L.LeakyReLU()(h)
    h = L.Dropout(rate)(h)
    return L.Add()([x, h])

def forward(x, unit, kernel = 3, rate = 0.1):
    h = L.Conv1D(unit, kernel, 1, padding = "same", activation = None)(x)
    h = L.LayerNormalization()(h)
    h = L.Dropout(rate)(h)
    h = L.LeakyReLU()(h)
    h = res(h, unit, kernel, rate)
    return h

def adj_attn(x, adj, unit, n = 2, rate = 0.1):
    x_a = x
    x_as = []
    for i in range(n):
        x_a = forward(x_a, unit)
        x_a = tf.linalg.matmul(adj, x_a)
        x_as.append(x_a)
    if n == 1:
        x_a = x_as[0]
    else:
        x_a = L.Concatenate()(x_as)
    x_a = forward(x_a, unit)
    return x_a


def get_base(config, dim=None):
    node = tf.keras.Input(shape = (dim, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (dim, dim, As.shape[3]), name = "adj")
    
    adj_learned = L.Dense(1, "relu")(adj)
    adj_all = L.Concatenate(axis = 3)([adj, adj_learned])
        
    xs = []
    xs.append(node)
    x1 = forward(node, 128*2, kernel = 3, rate = 0.1)
    x2 = forward(x1, 64*2, kernel = 6, rate = 0.1)
    x3 = forward(x2, 32*2, kernel = 15, rate = 0.1)
    x4 = forward(x3, 16*2, kernel = 30, rate = 0.1)
    x = L.Concatenate()([x1, x2, x3, x4])
    
    for unit in [64*2, 32*2]:
        x_as = []
        for i in range(adj_all.shape[3]):
            x_a = adj_attn(x, adj_all[:, :, :, i], unit, rate = 0)
            x_as.append(x_a)
        x_c = forward(x, unit, kernel = 30)
        
        x = L.Concatenate()(x_as + [x_c])
        x = forward(x, unit)
        x = multi_head_attention(x, x, unit, 4, 0.0)
        xs.append(x)
        
    x = L.Concatenate()(xs)

    model = tf.keras.Model(inputs = [node, adj], outputs = [x])
    return model


def get_ae_model(base, config, dim=None):
    node = tf.keras.Input(shape = (dim, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (dim, dim, As.shape[3]), name = "adj")

    x = base([L.SpatialDropout1D(0.3)(node), adj])
    x = forward(x, 64*2, rate = 0.2)
    p = L.Dense(X_node.shape[2], "sigmoid")(x)
    

    node_1 = tf.where((node>1-1e-8), node, tf.zeros_like(node))
    node_0 = tf.where((node<1e-8), node, tf.ones_like(node))
    node_float = tf.where((node<=1-1e-8)&(node>=1e-8), node, p) 
    
    loss = - tf.reduce_mean(20 * node_1 * tf.math.log(p + 1e-4) + (1 - node_0) * tf.math.log(1 - p + 1e-4) - 5*(node_float-p)**2)
    
    model = tf.keras.Model(inputs = [node, adj], outputs = [loss])
    
    opt = get_optimizer()
    model.compile(optimizer = opt, loss = lambda t, y : y)
    return model


def get_model(base, config, dim=None):
    node = tf.keras.Input(shape = (dim, X_node.shape[2]), name = "node")
    adj = tf.keras.Input(shape = (dim, dim, As.shape[3]), name = "adj")
    
    x = base([node, adj])
    if not Diversity_type in ['forward']:
        x = forward(x, 128*2, rate = 0.2)
    
    if Diversity_type == 'gru':
        x = gru_layer(128*2, dropout=0.2)(x)
    elif Diversity_type == 'lstm':
        x = lstm_layer(128*2, dropout=0.2)(x)
    elif Diversity_type == 'forward':
        x = forward(x, 128*4, kernel=5, rate = 0.1)
        x = forward(x, 128*4, kernel=3, rate = 0.1)
        x = forward(x, 128*4, kernel=1, rate = 0.1)
    elif Diversity_type == 'wave':
        dropout = 0.1
        x = wave_block(x, 16*2, 3, 12)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(dropout)(x)

        x = wave_block(x, 32*2, 3, 8)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(dropout)(x)

        x = wave_block(x, 64*2, 3, 4)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(dropout)(x)

        x = wave_block(x, 128*2, 3, 1)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(dropout)(x)
    
    
    x = x[:, 1:-1,:]
    x = L.Dense(5)(x)

    model = tf.keras.Model(inputs = [node, adj], outputs = [x])
    
    opt = get_optimizer()
    model.compile(optimizer = opt, loss = MSE(reduction=tf.keras.losses.Reduction.NONE))#mcrmse_loss)
    return model

def get_optimizer():
#     sgd = tf.keras.optimizers.SGD(0.05, momentum = 0.9, nesterov=True)
    adam = tf.optimizers.Adam()
#     radam = tfa.optimizers.RectifiedAdam()
#     lookahead = tfa.optimizers.Lookahead(adam, sync_period=6)
#     swa = tfa.optimizers.SWA(adam)
    return adam

In [14]:
## sequence
def return_ohe(n, i):
    tmp = [0] * n
    tmp[i] = 1
    return tmp

def get_input(train):
    
    len_app = 28
    seq_app = 'AGCUAGCUAGCUAGCUAGCUAGCUAGCU'
    loop_app = 'SSSSMMMMIIIIBBBBHHHHEEEEXXXX'
    stru_app = '.'*len_app
    
    train = train.copy()
    train['sequence'] = train['sequence'].apply(lambda x: x+seq_app)
    train['bpRNA_string'] = train['bpRNA_string'].apply(lambda x: x+loop_app)
    train['structure'] = train['structure'].apply(lambda x: x+stru_app)
    
    mapping = {}
    vocab = ["A", "G", "C", "U", "s", "e"]
    for i, s in enumerate(vocab):
        mapping[s] = return_ohe(len(vocab), i)
    X_node = np.stack(train["sequence"].apply(lambda x : list(map(lambda y : mapping[y], ['s']+list(x)+['e']))))

    mapping = {}
    vocab = ["S", "M", "I", "B", "H", "E", "X"]
    for i, s in enumerate(vocab):
        mapping[s] = return_ohe(len(vocab), i)
    X_loop = np.stack(train["bpRNA_string"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))
    X_loop = np.concatenate([np.zeros((X_loop.shape[0], 1, X_loop.shape[2])), X_loop, np.zeros((X_loop.shape[0], 1, X_loop.shape[2]))], axis=1)
    

    mapping = {}
    vocab = [".", "(", ")"]
    for i, s in enumerate(vocab):
        mapping[s] = return_ohe(len(vocab), i)
    X_structure = np.stack(train["structure"].apply(lambda x : list(map(lambda y : mapping[y], list(x)))))
    X_structure = np.concatenate([np.zeros((X_structure.shape[0], 1, X_structure.shape[2])), X_structure, np.zeros((X_structure.shape[0], 1, X_structure.shape[2]))], axis=1)
    
    
    X_node = np.concatenate([X_node, X_loop], axis = 2)
    
    ## interaction
    a = np.sum(X_node * (2 ** np.arange(X_node.shape[2])[None, None, :]), axis = 2)
    vocab = sorted(set(a.flatten()))
    #print(vocab)
    ohes = []
    for v in vocab:
        ohes.append(a == v)
    ohes = np.stack(ohes, axis = 2)
    X_node = np.concatenate([X_node, ohes], axis = 2).astype(np.float32)
    
    X_node = np.concatenate([X_node[:, :(-len_app-1), :], X_node[:, -1, :][:, None,:]], axis=1)
    #print(X_node.shape)
    return X_node

In [15]:
# copy and edited from https://www.kaggle.com/xhlulu/openvaccine-simple-gru-model

token2int = {x:i for i, x in enumerate('().ACGUBEHIMSXse')}
data_dir = '../../../DeepDeg3/data/'

def pandas_list_to_array(df):
    """
    Input: dataframe of shape (x, y), containing list of length l
    Return: np.array of shape (x, l, y)
    """
    
    return np.transpose(
        np.array(df.values.tolist()),
        (0, 2, 1)
    )

def get_pair_idx(arr, sft=0):
    n = len(arr)
    out = np.zeros((n))
    l = []
    for c, i in enumerate(arr):
        if i == '.':
            out[c] = c
        elif i == '(':
            l.append(c)
        else:
            temp = l.pop()
            if sft == 0:
                out[c] = temp
                out[temp] = c
            elif sft >= 1:
                out[c] = min(temp+sft, n-1)
                out[temp] = max(c-sft, 0)
            elif sft <= -1:
                out[c] = max(temp-sft, 0)
                out[temp] = min(c+sft, n-1)
    return out

def calc_dist_to_pair(struct):
    n = len(struct)
    out = np.zeros((n))+10000
    curr_dist = 10000
    for c,i in enumerate(struct):
        curr_dist += 1
        if i in ['(', ')']:
            out[c] = 1
            curr_dist = 0
        else:
            out[c] = min(out[c], curr_dist)
    curr_dist = 10000
    for c,i in enumerate(struct[::-1]):
        curr_dist += 1
        if i in ['(', ')']:
            out[n-1-c] = 0
            curr_dist = 0
        else:
            out[n-1-c] = min(out[n-1-c], curr_dist)
    return out


def calc_dist_to_single(struct):
    n = len(struct)
    out = np.zeros((n))+10000
    curr_dist = 10000
    for c,i in enumerate(struct):
        curr_dist += 1
        if i == '.':
            out[c] = 1
            curr_dist = 0
        else:
            out[c] = min(out[c], curr_dist)
    curr_dist = 10000
    for c,i in enumerate(struct[::-1]):
        curr_dist += 1
        if i == '.':
            out[n-1-c] = 0
            curr_dist = 0
        else:
            out[n-1-c] = min(out[n-1-c], curr_dist)
    return out


def preprocess_inputs1(df, token2int, cols=['sequence', 'structure', 'bpRNA_string']):
    return pandas_list_to_array(
        df[cols].applymap(lambda seq: [token2int[x] for x in 's'+seq+'e'])
    )
def preprocess_inputs(df, token2int):
    dict_row_idx = {}

    train_inputs = preprocess_inputs1(df, token2int)
    new = np.zeros((train_inputs.shape[0], train_inputs.shape[1], len(token2int)))
    for layer in range(3):
        for i in range(len(token2int)):
            new[train_inputs[:, :, layer]==i, i]=1

    if BBP_TOTAL>=1:
        bbp =[]
        bbp1 =[]
        bbp2 = []
        bbp3 = []
        bbp4_0 = []
        bbp4_1 = []
        bbp4_2 = []
        bbp4_3 = []

        ids = df.id.values
        for c, i in enumerate(ids):
            if 'aug' in str(i):
                probability = np.load(aug_data_path+f"bbps_aug/{i}.npy")
            else:
                f = gzip.GzipFile(data_dir+'Round6_3/bpps_eternafold_CDS/%s.npy.gz'%i, 'r')
                probability = np.load(f)
            if BBP:
                bbp.append(probability.max(-1).tolist())
            if BBP1:
                bbp1.append((1-probability.sum(axis=1)).tolist())
            if BBP2:
                srt = np.sort(probability)
                bbp2.append((srt[:,-1] - srt[:, -2]).tolist())
            if BBP3:
                m_lst = probability.max(axis=0)
                argmax_lst = m_lst[np.argmax(probability, axis=0)]
                bbp3.append((argmax_lst-m_lst).tolist())
            if BBP4:
                pair_idx = get_pair_idx(df.structure.values[c]).astype(int)
                pij = probability[np.arange(len(pair_idx)),pair_idx]
                bbp4_0.append(pij.tolist())
                m_lst = probability.max(axis=0)
                bbp4_1.append((m_lst-pij).tolist())
                bbp4_2.append((m_lst[pair_idx]-pij).tolist())
                s_lst = probability.sum(axis=0)
                bbp4_3.append((s_lst[pair_idx]-pij).tolist())

        temp = np.zeros((train_inputs.shape[0], train_inputs.shape[1]))
        if BBP:
            temp[:, 1:-1] = np.array(bbp)
            dict_row_idx['BBP'] = new.shape[2]
            new = np.concatenate([new, temp[:, :,None]], axis=2)
        if BBP1:
            temp[:, 1:-1] = np.array(bbp1)
            dict_row_idx['BBP1'] = new.shape[2]
            new = np.concatenate([new, temp[:, :,None]], axis=2)
        if BBP2:
            temp[:, 1:-1] = np.array(bbp2)
            dict_row_idx['BBP2'] = new.shape[2]
            new = np.concatenate([new, temp[:, :,None]], axis=2)
        if BBP3:
            temp[:, 1:-1] = np.array(bbp3)
            dict_row_idx['BBP3'] = new.shape[2]
            new = np.concatenate([new, temp[:, :,None]], axis=2)
        if BBP4: 
            for cnt, b in enumerate([bbp4_0, bbp4_1, bbp4_2, bbp4_3]):
                dict_row_idx['BBP4_%s'%cnt] = new.shape[2]
                temp[:, 1:-1] = np.array(b)
                new = np.concatenate([new, temp[:, :,None]], axis=2)
            dict_row_idx['BBP4_ed'] = new.shape[2]

            
    if DIST_NEW:
        lst_dist = []
        lst_dist_sqrt = []
        ids = df.id.values
        for c, i in enumerate(ids):
            temp_dist = calc_dist_to_pair(df['structure'].values[c])+1
            lst_dist.append((1/temp_dist).tolist())
            lst_dist_sqrt.append((np.sqrt(1/temp_dist)).tolist())
        temp = np.zeros((train_inputs.shape[0], train_inputs.shape[1]))
        temp[:, 1:-1] = np.array(lst_dist)
        new = np.concatenate([new, temp[:, :,None]], axis=2)
        temp = np.zeros((train_inputs.shape[0], train_inputs.shape[1]))
        temp[:, 1:-1] = np.array(lst_dist_sqrt)
        new = np.concatenate([new, temp[:, :,None]], axis=2)
        
    if DIST_NEW2:
        lst_dist = []
        lst_dist_sqrt = []
        ids = df.id.values
        for c, i in enumerate(ids):
            temp_dist = calc_dist_to_single(df['structure'].values[c])+1
            lst_dist.append((1/temp_dist).tolist())
            lst_dist_sqrt.append((np.sqrt(1/temp_dist)).tolist())
        temp = np.zeros((train_inputs.shape[0], train_inputs.shape[1]))
        temp[:, 1:-1] = np.array(lst_dist)
        new = np.concatenate([new, temp[:, :,None]], axis=2)
        temp = np.zeros((train_inputs.shape[0], train_inputs.shape[1]))
        temp[:, 1:-1] = np.array(lst_dist_sqrt)
        new = np.concatenate([new, temp[:, :,None]], axis=2)
    

    return new[:,:,len(token2int):]


In [16]:
def get_structure_adj(train):
    Ss = []
    for i in (range(len(train))):
        seq_length = train["seq_length"].iloc[i]
        structure = train["structure"].iloc[i]
        sequence = train["sequence"].iloc[i]

        cue = []
        a_structures = {
            ("A", "U") : np.zeros([seq_length, seq_length]),
            ("C", "G") : np.zeros([seq_length, seq_length]),
            ("U", "G") : np.zeros([seq_length, seq_length]),
            ("U", "A") : np.zeros([seq_length, seq_length]),
            ("G", "C") : np.zeros([seq_length, seq_length]),
            ("G", "U") : np.zeros([seq_length, seq_length]),
        }
        a_structure = np.zeros([seq_length, seq_length])
        for i in range(seq_length):
            if structure[i] == "(":
                cue.append(i)
            elif structure[i] == ")":
                start = cue.pop()
                a_structures[(sequence[start], sequence[i])][start, i] = 1
                a_structures[(sequence[i], sequence[start])][i, start] = 1
        
        a_strc = np.stack([a for a in a_structures.values()], axis = 2)
        a_strc = np.sum(a_strc, axis = 2, keepdims = True)
        Ss.append(a_strc)
    
    Ss = np.array(Ss)
    new = np.zeros((Ss.shape[0], Ss.shape[1]+2, Ss.shape[2]+2, Ss.shape[3]))
    new[:, 1:-1, 1:-1, :] = Ss
    return new

In [17]:
def get_distance_matrix(As):
    idx = np.arange(As.shape[1])
    Ds = []
    for i in range(len(idx)):
        d = np.abs(idx[i] - idx)
        Ds.append(d)

    Ds = np.array(Ds) + 1
    Ds = 1/Ds
    Ds = Ds[None, :,:]
    Ds = np.repeat(Ds, len(As), axis = 0)
    
    Dss = []
    for i in [1, 2, 4]:
        Dss.append(Ds ** i)
    Ds = np.stack(Dss, axis = 3)
    return Ds


def padding_2D(Ss):
    new = np.zeros((Ss.shape[0], Ss.shape[1]+2, Ss.shape[2]+2))
    new[:, 1:-1, 1:-1] = Ss
    return new

In [18]:
df.id.values

array([10421216, 10421202, 10421119, 10421010, 10420989, 10420977,
       10420831, 10420662, 10420583, 10420579, 10420550, 10420539,
       10419748, 10419746, 10419742, 10419578, 10419569, 10419565,
       10419549, 10419525, 10419522, 10419457, 10419441, 10419412,
       10419398, 10419396, 10419329, 10419159, 10419108, 10419092,
       10418991, 10418986, 10418947, 10418790, 10418785, 10418756,
       10418713, 10418630, 10418559, 10418506, 10418287, 10418259,
       10418242, 10418145, 10418067, 10417671, 10417629, 10417598,
       10417494, 10417486, 10417290, 10417205, 10417189, 10417131,
       10417129, 10417068, 10417051])

In [19]:
def get_inputs(df_temp):
    

    X_node = get_input(df_temp).astype(np.float32)
    X_node_new = preprocess_inputs(df_temp, token2int).astype(np.float32)
    X_node = np.concatenate([X_node, X_node_new], axis=2)
    del X_node_new


    As = []
    for ii in (df_temp["id"]):
        f = gzip.GzipFile(data_dir+'Round6_3/bpps_eternafold_CDS/%s.npy.gz'%ii, 'r')
        a = np.load(f)

        As.append(a)
    As = np.array(As)
    As = padding_2D(As)
    Ss = get_structure_adj(df_temp).astype(np.float32)
    Ds = get_distance_matrix(As)
    DDs = get_distance_matrix_2d(Ss)
    As = np.concatenate([As[:,:,:,None],Ss, Ds, DDs], axis = 3).astype(np.float32)
    del Ss, Ds, DDs
    return X_node, As

dict_X = {}
dict_A = {}
for i in tqdm(df.id.values):
    df_temp = df.loc[df.id == i]
    dict_X[i], dict_A[i] = get_inputs(df_temp)
    
#X_node, As = dict_X[0], dict_A[0]

100%|██████████| 57/57 [00:25<00:00,  2.22it/s]


In [20]:
X_node, As = dict_X[10421216], dict_A[10421216]


In [21]:
df.head()

,id,title,name,sequence,CAI,GC content,SUP vienna,SUP first 14 vienna,SUP first 30 vienna,punp_vec vienna,...,degscore,MLD,n_hairpins,n_3WJs,n_4WJs,n_5WJs_up,hp_3WJ_ratio,structure,bpRNA_string,seq_length
0,10421216,Mod of wateronthemoon's OV:nanofl 6,Merida,AUGGCCGUCUACCCCUACGACGUGCCCGACUACGCCGGCUACCCCU...,0.890295,59.742351,195.361171,5.711359,16.950537,[9.98375821e-01 6.18258391e-01 2.89686821e-03 ...,...,177.064,78,20,5,1,3,3.999999,..(((((((.........)))).))).......(((((((.(((((...,EESSSSSSSHHHHHHHHHSSSSBSSSEEEEEEESSSSSSSMSSSSS...,621
1,10421202,phoning it in 5,cynwulf28,AUGGCUGUAUAUCCGUAUGAUGUGCCCGAUUAUGCUGGGUACCCAU...,0.754168,51.529791,191.430001,3.476410,10.565406,[9.87866639e-01 9.84536094e-01 3.36349718e-03 ...,...,188.019,118,14,8,2,0,1.750000,..(((((.((((((((.....(((((((.......))))))).(((...,EESSSSSBSSSSSSSSMMMMMSSSSSSSHHHHHHHSSSSSSSMSSS...,621
2,10421119,Mod of DigitalEmbrace's Curevac 4 Mod 2-Kaggl...,Merida,AUGGCUGUUUACCCUUACGAUGUUCCCGACUACGCCGGGUACCCGU...,0.786049,59.742351,179.628614,7.173642,15.450940,[9.74989208e-01 9.99191615e-01 5.90287475e-03 ...,...,170.490,96,17,6,1,2,2.833333,..(((((((............(((((((.(...(.(((((((.(((...,EESSSSSSSMMMMMMMMMMMMSSSSSSSISIIISISSSSSSSISSS...,621
3,10421010,phoning it in 4,cynwulf28,AUGGCCGUGUAUCCUUAUGAUGUUCCAGAUUAUGCCGGCUACCCCU...,0.778542,49.758454,218.053576,5.990581,8.633370,[9.73084676e-01 9.34549226e-01 9.55890625e-01 ...,...,206.416,61,16,4,1,2,3.999999,...((((((((.(((..((((((((((((((((.(((((..........,EEESSSSSSSSBSSSIISSSSSSSSSSSSSSSSISSSSSIIIIIII...,621
4,10420989,phoning it in 3,cynwulf28,AUGGCCGUUUACCCGUACGAUGUGCCUGACUACGCGGGCUAUCCGU...,0.797147,55.877617,213.959940,3.269031,6.952990,[7.92954489e-01 6.56017560e-01 1.23806781e-01 ...,...,191.271,90,18,6,1,2,3.000000,(((((((((..((((((((....(((((......)))))....)))...,SSSSSSSSSIISSSSSSSSIIIISSSSSHHHHHHSSSSSIIIISSS...,621


In [22]:
%%time
config = {}
Diversity_type = 'lstm'
wgts_dir = '../../model_files/ov-v40032-wgts/'
base = get_base(config)
model = get_model(base, config)
for m in range(5):
    print(m)
    model.load_weights(wgts_dir+'model_%s.h5'%m)
    preds_ls = []
    for uid in tqdm(df.id.values):
        X_node, As = dict_X[uid], dict_A[uid]
        out1 = model.predict([X_node, As])
        out2 = model.predict([reverse_input(X_node), reverse_BBP_3D(As)])[:,::-1,:]
        out = (out1+out2)/2
        
        single_pred = out[0]
        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
        preds_ls.append(single_df)
        del out1, out2, out, single_pred, single_df
        
    preds_df = pd.concat(preds_ls).set_index('id_seqpos')
    preds_df.to_csv("sub_%s_%s.csv"%(Diversity_type, m))
    
    del preds_df, preds_ls
    gc.collect()
    gc.collect()


del base, model
gc.collect()

K.clear_session()

0


100%|██████████| 57/57 [02:22<00:00,  2.50s/it]


1


100%|██████████| 57/57 [01:50<00:00,  1.93s/it]


2


...

3


100%|██████████| 57/57 [01:56<00:00,  2.04s/it]


4


100%|██████████| 57/57 [01:53<00:00,  1.98s/it]


CPU times: user 2h 34min 26s, sys: 3h 34min 1s, total: 6h 8min 27s
Wall time: 10min 13s


In [23]:
import tensorflow as tf

In [24]:
tf.__version__

'2.4.1'

In [22]:
uid

10417051

In [25]:
%%time
config = {}
Diversity_type = 'gru'
wgts_dir = '../../model_files/ov-v40131-wgts/'
base = get_base(config)
model = get_model(base, config)
for m in range(5):
    model.load_weights(wgts_dir+'model_%s.h5'%m)
    preds_ls = []
    for uid in tqdm(df.id):
        X_node, As = dict_X[uid], dict_A[uid]
        out1 = model.predict([X_node, As])
        out2 = model.predict([reverse_input(X_node), reverse_BBP_3D(As)])[:,::-1,:]
        out = (out1+out2)/2
        
        single_pred = out[0]
        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
        preds_ls.append(single_df)
        del out1, out2, out, single_pred, single_df
        
    preds_df = pd.concat(preds_ls).set_index('id_seqpos')
    preds_df.to_csv("sub_%s_%s.csv"%(Diversity_type, m))
    
    del preds_df, preds_ls
    gc.collect()
    gc.collect()

del base, model
gc.collect()

K.clear_session()

100%|██████████| 57/57 [02:09<00:00,  2.28s/it]


CPU times: user 2h 42min 34s, sys: 3h 44min 55s, total: 6h 27min 29s
Wall time: 10min 41s


In [26]:
%%time
config = {}
Diversity_type = 'forward'
wgts_dir = '../../model_files/ov-v40237-wgts/'
base = get_base(config)
model = get_model(base, config)
for m in range(5):
    model.load_weights(wgts_dir+'model_%s.h5'%m)
    preds_ls = []
    for uid in tqdm(df.id):
        X_node, As = dict_X[uid], dict_A[uid]
        out1 = model.predict([X_node, As])
        out2 = model.predict([reverse_input(X_node), reverse_BBP_3D(As)])[:,::-1,:]
        out = (out1+out2)/2
        
        single_pred = out[0]
        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
        preds_ls.append(single_df)
        del out1, out2, out, single_pred, single_df
        
    preds_df = pd.concat(preds_ls).set_index('id_seqpos')
    preds_df.to_csv("sub_%s_%s.csv"%(Diversity_type, m))
    
    del preds_df, preds_ls
    gc.collect()
    gc.collect()

del base, model
gc.collect()

K.clear_session()

100%|██████████| 57/57 [01:17<00:00,  1.36s/it]


CPU times: user 1h 47min 48s, sys: 2h 23min 47s, total: 4h 11min 36s
Wall time: 7min 14s


In [27]:
%%time
config = {}
Diversity_type = 'wave'
wgts_dir = '../../model_files/ov-v40334-wgts/'
base = get_base(config)
model = get_model(base, config)
for m in range(5):
    model.load_weights(wgts_dir+'model_%s.h5'%m)
    preds_ls = []
    for uid in tqdm(df.id):
        X_node, As = dict_X[uid], dict_A[uid]
        out1 = model.predict([X_node, As])
        out2 = model.predict([reverse_input(X_node), reverse_BBP_3D(As)])[:,::-1,:]
        out = (out1+out2)/2
        
        single_pred = out[0]
        single_df = pd.DataFrame(single_pred, columns=pred_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]
        preds_ls.append(single_df)
        del out1, out2, out, single_pred, single_df
        
    preds_df = pd.concat(preds_ls).set_index('id_seqpos')
    preds_df.to_csv("sub_%s_%s.csv"%(Diversity_type, m))
    
    del preds_df, preds_ls
    gc.collect()
    gc.collect()
    
del base, model
gc.collect()

100%|██████████| 57/57 [01:16<00:00,  1.34s/it]


CPU times: user 1h 46min 47s, sys: 2h 24min 36s, total: 4h 11min 24s
Wall time: 7min 13s


449696

In [28]:
lst_pred = os.listdir()
lst_pred = sorted([x for x in lst_pred if x.startswith('sub_')])
print(lst_pred)
preds_df_agg = pd.read_csv(lst_pred[0], index_col=0)
for n in lst_pred[1:]:
    pred_temp = pd.read_csv(n, index_col=0)
    pred_temp[pred_temp<-0.5] = -0.5
    pred_temp[pred_temp>6] = 6
    preds_df_agg += pred_temp
preds_df_agg = preds_df_agg/len(lst_pred)
preds_df_agg = preds_df_agg.reset_index()

['sub_forward_0.csv', 'sub_forward_1.csv', 'sub_forward_2.csv', 'sub_forward_3.csv', 'sub_forward_4.csv', 'sub_gru_0.csv', 'sub_gru_1.csv', 'sub_gru_2.csv', 'sub_gru_3.csv', 'sub_gru_4.csv', 'sub_lstm_0.csv', 'sub_lstm_1.csv', 'sub_lstm_2.csv', 'sub_lstm_3.csv', 'sub_lstm_4.csv', 'sub_wave_0.csv', 'sub_wave_1.csv', 'sub_wave_2.csv', 'sub_wave_3.csv', 'sub_wave_4.csv']


In [29]:
preds_df_agg.shape

(35397, 6)

In [30]:
preds_df_agg

,id_seqpos,reactivity,deg_Mg_pH10,deg_Mg_50C,deg_pH10,deg_50C
0,10421216_0,0.770599,1.058450,0.891516,1.420543,0.838332
1,10421216_1,0.453072,0.798099,0.596542,0.980342,0.613319
2,10421216_2,0.117890,0.295279,0.157230,0.371076,0.243127
3,10421216_3,0.058547,0.207289,0.098000,0.267937,0.163007
4,10421216_4,0.018613,0.111981,0.058424,0.157092,0.079198
...,...,...,...,...,...,...
35392,10417051_616,0.173873,0.321040,0.329274,0.422107,0.426409
35393,10417051_617,0.575041,0.883880,0.899710,0.900326,0.820534
35394,10417051_618,0.280324,0.318513,0.368946,0.494258,0.491868
35395,10417051_619,0.482553,1.215590,1.180716,1.425635,1.207726


In [31]:
preds_df_agg.to_csv('preds_df_agg_57_nullrecurrent.csv', index=False)

In [32]:
preds_df_agg.columns

Index(['id_seqpos', 'reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10',
       'deg_50C'],
      dtype='object')

In [33]:
deg_pH10 = list(preds_df_agg['deg_pH10'].values)

In [34]:
len(deg_pH10)/621

57.0

In [35]:
n = 621 
chunked = [deg_pH10[i*n:(i+1)*n] for i in range(57)]

In [36]:
len(chunked[0])

621

In [37]:
sum_57 = [sum(chunked[j]) for j in range(57)]

In [38]:
for i in range(57):
    print(sum_57[i])

211.408297225025
217.60565871987995
203.00585398206437
222.51394208472507
218.08743832610156
222.46819609496495
179.83827949167514
195.21370454126358
196.66175301240003
196.29407831152997
157.96437488743987
181.2577580721414
236.61675636165015
234.47192957321496
231.98164372436983
158.85080422896803
152.57500735695174
163.4670522440799
190.07416487438496
155.63861850649988
192.92181547135982
170.68993148681673
191.85518348716994
190.35875913231595
159.2667865243951
205.69007985199997
168.0030830956424
221.08251122812024
236.07649804499476
168.53180565278666
233.29345849965006
194.67901231235
197.10182969757003
169.26588951552
197.3668855286899
173.18569995465498
173.04882611499286
152.23302971085513
231.1927633417452
203.45687194801513
203.14619762789494
197.24593657440013
162.91199499673493
200.94699041354997
163.32688459159473
221.98922996811018
230.17670457091955
182.50422532468
238.42478240361302
157.95593264644552
212.4648267736446
212.4594815333525
194.2440188868251
198.449850337

In [39]:
with open("nullrecurrent_deg_1day_pH10_57_preds_1.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(chunked)

In [40]:
nullrecurrent_deg_1day_pH10_57_preds_2 = pd.DataFrame(data=chunked, columns=[f'col_{i}' for i in range(621)])
nullrecurrent_deg_1day_pH10_57_preds_2.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_611,col_612,col_613,col_614,col_615,col_616,col_617,col_618,col_619,col_620
0,1.420543,0.980342,0.371076,0.267937,0.157092,0.161060,0.336868,0.165699,0.357519,0.354406,...,0.216735,0.207147,0.079235,0.173797,0.153577,0.144213,0.270501,0.366645,1.865139,0.974362
1,1.566976,1.065174,0.392607,0.265059,0.271024,0.200612,0.699897,1.269791,0.589301,0.228388,...,0.295258,0.201656,0.135037,0.217441,0.179295,0.156741,0.325050,0.678433,1.131142,1.830353
2,1.122458,1.123113,0.360840,0.223545,0.207097,0.161379,0.754339,0.692215,0.605911,0.865006,...,0.505988,0.321021,0.526941,0.414577,0.334630,0.648210,0.719522,0.901765,1.996982,1.159678
3,1.587663,0.974604,0.943554,0.412973,0.182546,0.199072,0.315867,0.348984,1.022649,0.257710,...,0.599387,0.321707,0.269786,0.421521,0.355287,0.304744,0.693773,0.470673,1.406758,0.925665
4,1.893198,0.999041,0.548853,0.337310,0.158981,0.168329,0.667319,0.745723,0.596992,0.511314,...,0.488991,0.938099,0.275175,0.611998,0.273924,0.312864,0.826558,1.277343,1.039246,0.821305


In [41]:
nullrecurrent_deg_1day_pH10_57_preds_2['id'] = df['id']
nullrecurrent_deg_1day_pH10_57_preds_2['sequence'] = df['sequence']
nullrecurrent_deg_1day_pH10_57_preds_2['bprna_string'] = df['bprna_string']
nullrecurrent_deg_1day_pH10_57_preds_2['nullrecurrent_sum'] = sum_57
nullrecurrent_deg_1day_pH10_57_preds_2.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_615,col_616,col_617,col_618,col_619,col_620,id,sequence,bprna_string,nullrecurrent_sum
0,1.420543,0.980342,0.371076,0.267937,0.157092,0.161060,0.336868,0.165699,0.357519,0.354406,...,0.153577,0.144213,0.270501,0.366645,1.865139,0.974362,10421216,AUGGCCGUCUACCCCUACGACGUGCCCGACUACGCCGGCUACCCCU...,EESSSSSSSHHHHHHHHHSSSSBSSSEEEEEEESSSSSSSMSSSSS...,211.408297
1,1.566976,1.065174,0.392607,0.265059,0.271024,0.200612,0.699897,1.269791,0.589301,0.228388,...,0.179295,0.156741,0.325050,0.678433,1.131142,1.830353,10421202,AUGGCUGUAUAUCCGUAUGAUGUGCCCGAUUAUGCUGGGUACCCAU...,EESSSSSBSSSSSSSSMMMMMSSSSSSSHHHHHHHSSSSSSSMSSS...,217.605659
2,1.122458,1.123113,0.360840,0.223545,0.207097,0.161379,0.754339,0.692215,0.605911,0.865006,...,0.334630,0.648210,0.719522,0.901765,1.996982,1.159678,10421119,AUGGCUGUUUACCCUUACGAUGUUCCCGACUACGCCGGGUACCCGU...,EESSSSSSSMMMMMMMMMMMMSSSSSSSISIIISISSSSSSSISSS...,203.005854
3,1.587663,0.974604,0.943554,0.412973,0.182546,0.199072,0.315867,0.348984,1.022649,0.257710,...,0.355287,0.304744,0.693773,0.470673,1.406758,0.925665,10421010,AUGGCCGUGUAUCCUUAUGAUGUUCCAGAUUAUGCCGGCUACCCCU...,EEESSSSSSSSBSSSIISSSSSSSSSSSSSSSSISSSSSIIIIIII...,222.513942
4,1.893198,0.999041,0.548853,0.337310,0.158981,0.168329,0.667319,0.745723,0.596992,0.511314,...,0.273924,0.312864,0.826558,1.277343,1.039246,0.821305,10420989,AUGGCCGUUUACCCGUACGAUGUGCCUGACUACGCGGGCUAUCCGU...,SSSSSSSSSIISSSSSSSSIIIISSSSSHHHHHHSSSSSIIIISSS...,218.087438


In [42]:
nullrecurrent_deg_1day_pH10_57_preds_2.to_csv('nullrecurrent_deg_1day_pH10_57_preds_2.csv', index=False)